# DE27IM Class Tests

When written in binary, the 27 bit relationship contains 3 9-bit
parts corresponding to DE-9IM relationships. The left-most 9 bits
are the relationship between the second structure's contour and the
first structure's convex hull polygon. The middle 9 bits are the
relationship between the second structure's contour and the first
structure's exterior polygon (i.e. with any holes filled). The
right-most 9 bits are the relationship between the second
structure's contour and the first structure's contour.

Args:
    slice_structures (pd.DataFrame): A table of structures, where
        the values are the contours with type StructureSlice. The
        column index contains the roi numbers for the structures.
        The row index contains the slice index distances.

Returns:
    DE9IM_Type: An integer corresponding to a 27 bit binary value
        reflecting the combined DE-9IM relationship between contour2 and
        contour1's convex hull, exterior and polygon.

## Setup

### Imports

In [1]:
# Add src and Tests to sys.path for notebook imports
import sys
from pathlib import Path

root = Path.cwd().parent.parent.parent # Adjust if needed
src_path = root / "src"
tests_path = root / "Tests"
src_path

WindowsPath("d:/OneDrive - Queen's University/src")

In [2]:

if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))
if str(tests_path) not in sys.path:
    sys.path.insert(0, str(tests_path))

In [3]:
# Shared Packages
import pandas as pd

# Local functions and classes
# Local functions and classes
from types_and_classes import *
from utilities import *
from contours import *
from contour_graph import *
from debug_tools import *
from region_slice import *
from relations import *

### Global Settings

In [4]:
PRECISION = 2

In [5]:
%matplotlib inline

In [ ]:
def surrounds_example2():
    circle6 = shapely.Polygon(circle_points(3))
    circle5 = shapely.Polygon(circle_points(2.5))
    circle4 = shapely.Polygon(circle_points(2))
    circle3 = shapely.Polygon(circle_points(1.5))
    circle2 = shapely.Polygon(circle_points(1))

    a = StructureSlice([circle6, circle5, circle2])
    b = StructureSlice([circle4, circle3])
    return a, b

In [ ]:
def shelters_example():
    circle6 = shapely.Polygon(circle_points(3))
    circle5 = shapely.Polygon(circle_points(2.5))
    circle4_offset = shapely.Polygon(circle_points(2, offset_x=3.5))
    shell = shapely.difference(circle6, circle5)
    cove = shapely.difference(shell, circle4_offset)
    circle2 = shapely.Polygon(circle_points(1, offset_x=1))

    a = StructureSlice([cove])
    b = StructureSlice([circle2])
    return a, b

a, b = shelters_example()
ax = plot_ab(a.contour, b.contour)

r = DE27IM(a,b)
print(str(r))
print(r.identify_relation())

In [ ]:
poly_a = shapely.Polygon(circle_points(2))
poly_b = shapely.Polygon(circle_points(2))
relation = DE9IM(poly_a, poly_a)
print(relation)
relation

In [ ]:
relation_str = '111000000111000000111000000'
relation_str[9:]

In [ ]:
box6 = shapely.MultiPolygon([shapely.Polygon(box_points(6))])
box4 = shapely.MultiPolygon([shapely.Polygon(box_points(4))])
relation = DE9IM(box4, box6)
boundary_relation = relation.boundary_adjustment('b')
print(boundary_relation)
boundary_relation

In [ ]:
relation_str = '111000000111000000111000000'
de27im = DE27IM(relation_str=relation_str)

In [ ]:
DE27IM.test_binaries[7].relation_type


In [ ]:
v = DE27IM.test_binaries[7].mask - DE27IM.test_binaries[7].value
v = DE27IM.test_binaries[7].value
de27im = DE27IM(relation_int=v)
de27im.identify_relation()

## Adjusting Boundary Slice Relations

For the beginning and ending slices of a structure the entire contour must
be treated as a boundary.  The structure does not have an interior on these
slices. In this case the “*Interior*” relations become “*Boundary*” relations.

The “*Interior*” bits of the DE-9IM relationship metric are combined with the 
“*Boundary*” bits using "OR" logic.  This keeps the “*Boundary*” bits as 
part of the boundary.

Possible boundary adjustment types are:

<link rel="stylesheet" href="relations.css">
<ul>
<li>The <span class="a">a</span> polygon is from a boundary slice</li>
<li>The <span class="b">b</span> polygon is from a boundary slice</li>
<li>Both the <span class="a">a</span> polygon and 
<span class="b">b</span> polygons are from a boundary slice</li>
</ul>


In [ ]:
def surrounds_example2():
    circle6 = shapely.Polygon(circle_points(3))
    circle5 = shapely.Polygon(circle_points(2.5))
    circle4 = shapely.Polygon(circle_points(2))
    circle3 = shapely.Polygon(circle_points(1.5))
    circle2 = shapely.Polygon(circle_points(1))

    a = StructureSlice([circle6, circle5, circle2])
    b = StructureSlice([circle4, circle3])
    return a, b

In [ ]:
a, b = surrounds_example2()
plot_ab(a.contour, b.contour)
contour = a.contour.relate(b.contour)
external = a.exterior.relate(b.contour)
convex_hull = a.hull.relate(b.contour)
relation_group = (contour, external, convex_hull)
print(relation_group)

**Original**<br>
|Contour|Exterior|Hull|
|-----|-----|-----|
| FF2 | 212 | 212 |
| FF1 | FF1 | FF1 |
| 212 | FF2 | FF2 |

`('FF2FF1212','212FF1FF2','212FF1FF2')`<br>


**Boundary A Adjustment**<br>
<img src="../../Images/Boundaries/a interior to boundary.png" style="height:150px;"/><br>
|Contour|Exterior|Hull|
|-----|-----|-----|
| FFF | FFF | FFF |
| FF2 | 212 | 212 |
| 212 | FF2 | FF2 |


`('FFFFF2212','FFF212FF2','FFF212FF2')`<br>
(<DE9IM>: FFFFF2212, <DE9IM>: FFF212FF2, <DE9IM>: FFF212FF2)

**Boundary B Adjustment**<br>
<img src="../../Images/Boundaries/b interior to boundary.png" style="height:150px;"/><br>
|Contour|Exterior|Hull|
|-----|-----|-----|
| FF2 | F22 | F22 |
| FF1 | FF1 | FF1 |
| F22 | FF2 | FF2 |

`('FF2FF1F22','F22FF1FF2','F22FF1FF2')`<br>


**Both Boundary Adjustments**<br>
<img src="../../Images/Boundaries/a and b interior to boundary.png" style="height:150px;"/><br>
|Contour|Exterior|Hull|
|-----|-----|-----|
| FFF | FFF | FFF |
| FF2 | F22 | F22 |
| F22 | FF2 | FF2 |

`('FFFFF2F22','FFFF22FF2','FFFF22FF2')`<br>


In [ ]:
initial_relation_group = (DE9IM(relation_str='FF2FF1212'),
                          DE9IM(relation_str='212FF1FF2'),
                          DE9IM(relation_str='212FF1FF2'))
print(f'Initial Relation:\t\t\t{initial_relation_group}')

boundary_a_adjustment = (DE9IM(relation_str='FFFFF2212'),
                         DE9IM(relation_str='FFF212FF2'),
                         DE9IM(relation_str='FFF212FF2'))

print(f'After Boundary A Adjustment:\t\t{boundary_a_adjustment}')

boundary_b_adjustment = (DE9IM(relation_str='FF2FF1F22'),
                         DE9IM(relation_str='F22FF1FF2'),
                         DE9IM(relation_str='F22FF1FF2'))
print(f'After Boundary B Adjustment:\t\t{boundary_b_adjustment}')

both_boundary_adjustments = (DE9IM(relation_str='FFFFF2F22'),
                             DE9IM(relation_str='FFFF22FF2'),
                             DE9IM(relation_str='FFFF22FF2'))
print(f'After Both Boundary Adjustments:\t{both_boundary_adjustments}')


In [ ]:
de27im = DE27IM(relation_int=0)
adjusted_de27im = de27im.apply_adjustments(initial_relation_group,
                                           ['boundary_a'])
boundary_a_adjustment = (DE9IM(relation_str='FFFFF2212'),
                         DE9IM(relation_str='FFF212FF2'),
                         DE9IM(relation_str='FFF212FF2'))
print(adjusted_de27im)
print(boundary_a_adjustment)
assert adjusted_de27im == boundary_a_adjustment

In [ ]:
de27im = DE27IM(relation_int=0)
adjusted_de27im = de27im.apply_adjustments(initial_relation_group,
                                           ['boundary_b'])
boundary_b_adjustment = (DE9IM(relation_str='FF2FF1F22'),
                         DE9IM(relation_str='F22FF1FF2'),
                         DE9IM(relation_str='F22FF1FF2'))
print(adjusted_de27im)
print(boundary_b_adjustment)
assert adjusted_de27im == boundary_b_adjustment

In [ ]:
de27im = DE27IM(relation_int=0)
adjusted_de27im = de27im.apply_adjustments(initial_relation_group,
                                           ['boundary_a', 'boundary_b'])
both_boundary_adjustments = (DE9IM(relation_str='FFFFF2F22'),
                             DE9IM(relation_str='FFFF22FF2'),
                             DE9IM(relation_str='FFFF22FF2'))
print(adjusted_de27im)
print(both_boundary_adjustments)
assert adjusted_de27im == both_boundary_adjustments

## Correcting for Holes

In some cases a relation may be generated for a polygon that is represents a hole (void) in a larger polygon. 

In this case,  the the interior of a hole is the exterior of the structure, but 
the exterior of a hole is undefined. It may or may not be within the interior 
of the larger polygon.

- “*Interior*” relations become “*Exterior*” relations 
- “*Exterior*” relations become “**F**” relations.

- When hole boundaries are tested, only the "contour" “*Boundary*” bits of
    the `DE27IM` relationship are used.  The external and hull bits are set to `'FFFFFFFFF'`.

In [ ]:
def shelters_circle():
    circle6 = shapely.Polygon(circle_points(3))
    circle3 = shapely.Polygon(circle_points(1.5, offset_x=1.6))
    crescent = shapely.difference(circle6, circle3)
    circle2 = shapely.Polygon(circle_points(1, offset_x=1.5))

    a = StructureSlice([crescent])
    b = StructureSlice([circle2])
    return a, b

In [ ]:
a, b = shelters_circle()
plot_ab(a.contour, b.contour)
contour = a.contour.relate(b.contour)
external = a.exterior.relate(b.contour)
convex_hull = a.hull.relate(b.contour)
relation_group = (contour, external, convex_hull)
print(relation_group)

**Original**<br>
|Contour|Exterior|Hull|
|-----|-----|-----|
| FF2 | FF2 | 212 |
| FF1 | FF1 | FF1 |
| 212 | 212 | FF2 |

`('FF2FF1212','FF2FF1212','212FF1FF2')`<br>

**Hole A Adjustment**<br>
<img src="..\..\Images\Boundaries\Correction for A as Hole.png" style="height:150px;"/><br>
|Contour|Exterior|Hull|
|-----|-----|-----|
| FFF | FFF | FFF |
| FF1 | FFF | FFF |
| FF2 | FFF | FFF |

**Note:** The exterior and hull relations are not used for hole adjustments.

`('FFFFFFF2','FFFFFFFFF','FFFFFFFFF')`<br>
`('FFFFF1FF2','FFFFFFFFF','FFFFFFFFF')`<br>


**Hole B Adjustment**<br>
<img src="../../Images/Boundaries/b interior to boundary.png" style="height:150px;"/><br>
|Contour|Exterior|Hull|
|-----|-----|-----|
| FFF | FFF | FFF |
| FFF | FFF | FFF |
| F12 | FFF | FFF |

`('FFFFFFF12','FFFFFFFFF','FFFFFFFFF')`<br>


In [ ]:
initial_relation_group = (DE9IM(relation_str='FF2FF1212'),
                          DE9IM(relation_str='FFFFFFFFF'),
                          DE9IM(relation_str='FFFFFFFFF'))

de27im = DE27IM(relation_int=0)
adjusted_de27im = de27im.apply_adjustments(initial_relation_group,
                                           ['hole_a'])
hole_a_adjustment = (DE9IM(relation_str='FFFFF1FF2'),
                     DE9IM(relation_str='FFFFFFFFF'),
                     DE9IM(relation_str='FFFFFFFFF'))
print(adjusted_de27im)
print(hole_a_adjustment)
assert adjusted_de27im == hole_a_adjustment

In [ ]:
initial_relation_group = (DE9IM(relation_str='FF2FF1212'),
                          DE9IM(relation_str='FFFFFFFFF'),
                          DE9IM(relation_str='FFFFFFFFF'))

de27im = DE27IM(relation_int=0)
adjusted_de27im = de27im.apply_adjustments(initial_relation_group,
                                           ['hole_b'])
hole_b_adjustment = (DE9IM(relation_str='FFFFFFF12'),
                     DE9IM(relation_str='FFFFFFFFF'),
                     DE9IM(relation_str='FFFFFFFFF'))
print(adjusted_de27im)
print(hole_b_adjustment)
assert adjusted_de27im == hole_b_adjustment

## Transposed Relations

Currently, relationships are defined and tested assuming that the first 
   structure is the same size or larger in volume than the second structure.
   However, when testing boundaries, the first contour is assumed to be on a 
   boundary slice.  If the boundary being tested is a boundary of the second
    structure, the relationship must be transposed before being combined with 
    the other relation values.

> `shapely.relate(poly_a, poly_b)` $\equiv$ `shapely.relate(poly_a, poly_b).T`


In [ ]:
def simple_contains_example():
    circle6 = shapely.Polygon(circle_points(3))
    circle4 = shapely.Polygon(circle_points(2))
    a = StructureSlice([circle6])
    b = StructureSlice([circle4])
    return a, b


In [ ]:
a, b = simple_contains_example()
plot_ab(a.contour, b.contour)
contour = a.contour.relate(b.contour)
external = a.exterior.relate(b.contour)
convex_hull = a.hull.relate(b.contour)
relation_group = (contour, external, convex_hull)
print(relation_group)

**Original**<br>
|Contour|Exterior|Hull|
|-----|-----|-----|
| 212 | 212 | 212 |
| FF1 | FF1 | FF1 |
| FF2 | FF2 | FF2 |

`('212FF1FF2', '212FF1FF2', '212FF1FF2')`<br>

**Transpose**<br>
|Contour|Exterior|Hull|
|-----|-----|-----|
| 2FF | 2FF | 2FF |
| 1FF | 1FF | 1FF |
| 212 | 212 | 212 |


`('2FF1FF212', '2FF1FF212', '2FF1FF212')`<br>


In [ ]:
initial_relation_group = (DE9IM(relation_str='212FF1FF2'),
                          DE9IM(relation_str='212FF1FF2'),
                          DE9IM(relation_str='212FF1FF2'))

de27im = DE27IM(relation_int=0)
adjusted_de27im = de27im.apply_adjustments(initial_relation_group,
                                           ['transpose'])
transpose_adjustment = (DE9IM(relation_str='2FF1FF212'),
                        DE9IM(relation_str='2FF1FF212'),
                        DE9IM(relation_str='2FF1FF212'))
print(adjusted_de27im)
print(transpose_adjustment)
assert adjusted_de27im == transpose_adjustment

In [ ]:
def confines_ring_example():
    circle6 = shapely.Polygon(circle_points(3))
    circle4 = shapely.Polygon(circle_points(2))
    circle2 = shapely.Polygon(circle_points(1))

    # b has internal borders with the ring portion of a, but has an external
    # border with the island part of a. The internal borders relation wins.
    a = StructureSlice([circle6, circle4, circle2])
    b = StructureSlice([circle4, circle2])
    return a, b

In [ ]:
a, b = confines_ring_example()
plot_ab(a.contour, b.contour)
contour = a.contour.relate(b.contour)
external = a.exterior.relate(b.contour)
convex_hull = a.hull.relate(b.contour)
relation_group = (contour, external, convex_hull)
print(relation_group)

('
 FF2
 F11
 2F2
 
 ', '
 
 212
 FF1
 FF2
 
 ', '
 
 212
 FF1
 FF2
 
 ')

('
FF2
F11
2F2
', '
212
FF1
FF2
', '
212
FF1
FF2
')

In [ ]:
a, b = surrounds_example2()
plot_ab(a.contour, b.contour)
contour = a.contour.relate(b.contour)
external = a.exterior.relate(b.contour)
convex_hull = a.hull.relate(b.contour)
relation_group = (contour, external, convex_hull)
print(relation_group)

In [ ]:
a, b = shelters_circle()
plot_ab(a.contour, b.contour)
contour = a.contour.relate(b.contour)
external = a.exterior.relate(b.contour)
convex_hull = a.hull.relate(b.contour)
relation_group = (contour, external, convex_hull)
print(relation_group)